In [1]:
#*--coding utf-8 --*
%run -i modulo_busca_arquivos.py
from ipywidgets import widgets, Box, VBox, HBox, Button, Layout, Checkbox, Text, Label
from IPython.display import HTML, Javascript, display

css = """
<style> 
.container{
        width: 98%;
    }
.output output_scroll{
    height: 100%;
    }
.widget-label{
        heigth: 100px;
        white-space: wrap;
        font-weight: bold;
        font-size: 16px;
        text-align: center;
        }
.space{
        text-align: left;
        font-size: 13px;
    }
.vertical-line{
    border-right: 1px groove black;
    }
.horizontal-line{
    border-bottom: 1px groove black;
    }
</style>"""

emExibicao = widgets.Label(value="Exibindo "+str(len(t))+" de "+str(len(t))+" arquivos", description="Quantidade de Arquivos")
dev = []
train = []
decay = []
lr = []
hs = []
uniques = list()
filtro = dict()
cells_visable = True

def toggle_code_cells(btn):
    global cells_visable
    if cells_visable:
        display(Javascript("$('div.input').hide();"))
        btn.description = "Mostrar Celulas"
    else:
        display(Javascript("$('div.input').show();"))
        btn.description = "Esconder Celulas"
    cells_visable = not cells_visable
    
toggle_cell = widgets.Button(description="Esconder Celulas", button_style="danger", layout=Layout(align_self="center", display="flex-wrap"))
toggle_cell.on_click(toggle_code_cells)

def recolheMenu(n):
    if(left_box.layout.visibility == "hidden"):
        vBox.layout.width = "85%"
        left_box.layout.visibility = "visible"
        left_box.layout.width = "25%"
    else:
        left_box.layout.visibility = "hidden"
        left_box.layout.width = "0%"
        vBox.layout.width = "100%"
           
toggle_filter = widgets.Button(description="Esconder Filtros", button_style="info", layout=Layout(align_self="center", display="flex-wrap"))
toggle_filter.on_click(recolheMenu)

def aplicaFiltros(n):
    global filtro
    global emExibicao
    for l in lr:
        if l.value:
             filtro['lr'] = l.description
    for d in dev:
        if d.value:
            filtro['dev'] = d.description 
    for k in train:
        if k.value:
            filtro['train'] = k.description
    for h in hs:
        if h.value:
            filtro['hidden_size'] = h.description
    for dc in decay:
        if dc.value:
            filtro['decay'] = dc.description
    
    if len(filtro) != 0:
        emExibicao = widgets.Label(value="Exibindo "+str(len(filtro))+" de "+str(len(t))+" arquivos", description="Quantidade de Arquivos")
    
filter_btn = widgets.Button(description="Aplicar Filtros", button_style="primary", layout=Layout(align_self="flex-end", display = "flex-wrap"))
filter_btn.on_click(aplicaFiltros)

box_layout = Layout(width="100%", 
                    display="flex",
                    flex_direction="column",
                    border="2px groove grey",
                    align_items='stretch')

vBox_layout = Layout(width="85%",
                    align_content="flex-start",
                    display="flex")

left_layout = Layout(width="25%", height="95%", flex_flow="column", display="flex", align_items="flex-start", align_self="center")
right_layout = Layout(width="99%", flex_flow="row", justify_content="space-between", display="flex", align_items="flex-end", align_self="center")
main_layout = Layout(width="99%", height="100%")

filter_btn = widgets.Button(description="Aplicar Filtros", button_style="primary", layout=Layout(align_self="flex-end", display = "flex-wrap"))
filter_btn.on_click(aplicaFiltros)

box_layout = Layout(width="100%", 
                    display="flex",
                    flex_direction="column",
                    border="2px groove grey",
                    align_items='stretch')

vBox_layout = Layout(width="85%",
                    align_content="flex-start",
                    display="flex")

left_layout = Layout(width="25%", height="95%", flex_flow="column", display="flex", align_items="flex-start", align_self="center")
right_layout = Layout(width="99%", height="6%", flex_flow="row", justify_content="space-between", display="flex", align_items="flex-end", align_self="center")
main_layout = Layout(width="99%", height="94%")

def buildUniKey(key):
    for i in key:
        print i
        if cmp(i, 'lr') == 0:
            buildLr('u')
        elif cmp(i, 'train') == 0:
            buildTrain('u')
        elif cmp(i, 'hidden_size') == 0:
            buildHs('u')
        elif cmp(i, 'dev') == 0:
            buildDev('u')
        elif cmp(i, 'decay') == 0:
            buildDecay('u')

def buildMultKey(key):
    for i in key:
        if cmp(i, 'lr') == 0:
            buildLr('m')
        elif cmp(i, 'train') == 0:
            buildTrain('m')
        elif cmp(i, 'hidden_size') == 0:
            buildHs('m')
        elif cmp(i, 'dev') == 0:
            buildDev('m')
        elif cmp(i, 'decay') == 0:
            buildDecay('m')

def buildLr(tipo):
    global lr
    if cmp(tipo, 'u') == 0:
        lr = [widgets.Label(description=str(t['unique']['lr']), value="Learnig Rate: "+str(t['unique']['lr']),
                       width="95%",
                       style={'description_width': 'initial'}).add_class('space')]
    elif cmp(tipo, 'm') == 0:
        lr = [Checkbox(description=str(l), 
                       width="100%",
                       style={'description_width': 'initial'}) for l in t['multiple']['lr']]

def buildTrain(tipo):
    global train
    if cmp(tipo, 'u') == 0:
        train = [widgets.Label(description=str(t['unique']['train']), value="Train: "+str(t['unique']['train']),
                          width="95%",
                          style={'description_width': 'initial'}).add_class('space')]
        
    elif cmp(tipo, 'm') == 0:
        train = [Checkbox(description=r,
                          width="100%",
                          style={'description_width': 'initial'}) for r in t['multiple']['train']]
        
def buildHs(tipo):
    global hs
    if cmp(tipo, 'u') == 0:
        hs = [widgets.Label(description=str(t['unique']['hidden_size']), value="Hidden_Size: "+str(t['unique']['hidden_size']),
                       width="95%",
                       style={'description_width': 'initial'}).add_class('space')]
        
    elif cmp(tipo, 'm') == 0:
        hs = [Checkbox(description=str(z),
                       width="100%",
                       style={'description_width': 'initial'}) for z in t['multiple']['hidden_size']]

def buildDev(tipo):
    global dev
    if cmp(tipo, 'u') == 0:
        dev = [widgets.Label(description=t['unique']['dev'], value="Dev: "+t['unique']['dev'], width="95%",
                        style={'description_width': 'initial'}).add_class('space').add_class('space')]
        
    elif cmp(tipo, 'm') == 0:
        dev = [Checkbox(description=w,
                        width="100%",
                        style={'description_width': 'initial'}) for w in t['multiple']['dev']]
        
def buildDecay(tipo):
    global decay
    if cmp(tipo, 'u') == 0:
        decay = [widgets.Label(value="Decay: "+t['unique']['decay'], description=t['unique']['decay'], width="95%").add_class('space')]
        
    elif cmp(tipo, 'm') == 0:
        decay = [Checkbox(description=w,
                        width="100%",
                        style={'description_width': 'initial'}) for w in t['multiple']['decay']]


uniKey = t['unique'].keys()
multKey = t['multiple'].keys()
if len(uniKey) > 0:
    buildUniKey(uniKey)
        
if len(multKey) > 0:
    buildMultKey(multKey)
        
def getUnique():
    global uniques
    if len(decay) == 1:
        uniques.extend(decay)
    if len(lr) == 1:
        uniques.extend(lr)
    if len(dev) == 1:
        uniques.extend(dev)
    if len(train) == 1:
        uniques.extend(train)
    if len(hs) == 1:
        uniques.extend(hs)
    
    return uniques

unicos = getUnique()
if len(unicos) == 0:
    vazio = widgets.Label(value="Nao Ha", description="Nao ha", width="100%")
    unicos.append(vazio)
    

static = HBox([i for i in unicos],layout=Layout(width="100%", 
                               flex_flow="column",
                               display="inline-flex"))

if not 'lr' in unicos:
    first = HBox([i for i in lr], layout=Layout(width="100%",
                                                flex_flow="column",
                                                display="inline-flex"))
else:
    first = HBox([])
    
if not 'train' in unicos:
    second = HBox([i for i in train], layout=Layout(width="100%",
                                                    flex_flow="column",
                                                    display="flex-shrink"))
else:
    second = HBox([])
    
if not 'hidden_size' in unicos:
    third = HBox([i for i in hs], layout=Layout(width="100%",
                                                flex_flow="column",
                                                display="flex-shrink"))

else:
    third = HBox([])
    
if not 'dev' in unicos:
    fourth = HBox([i for i in dev], layout=Layout(width="100%",
                                                  flex_flow="column",
                                                  display="flex-shrink")) 

else:
    fourth = HBox([])
    
if not 'decay' in unicos:
    fifth = HBox([[i for i in decay]], layout=Layout(width="100%",
                                                  flex_flow="column",
                                                  display="flex-shrink")) 

else:
    fifth = HBox([])
# lab = ;
# lab.add_class("space");

left_box = VBox([widgets.Label(value="Filtros", description="Filtros", display="flex-shrink", layout=Layout(width="100%", align_self="center")),
                 widgets.Label(value="Caracteristicas Unicas", description="Caracteristicas Unicas", display="flex-shrink", layout=Layout(width="100%", align_self="center")),
                 static,
                 widgets.Label(value="Caracteristicas Multiplas", description="Caracteristicas Multiplas", display="flex-shrink", layout=Layout(width="100%", align_self="center")),
                 widgets.Label(value="Learning Rate", description="Learnig Rate", display="flex-shrink", layout=Layout(width="95%", align_self="center")).add_class("space"),
                 first,
                 widgets.Label(value="Train", description="Train", display="flex-shrink", layout=Layout(width="95%", align_self="center")).add_class("space"),
                 second,
                 widgets.Label(value="Hidden Size", description="Hidden Size", display="flex-shrink", layout=Layout(width="95%", align_self="center")).add_class("space"),
                 third,
                 widgets.Label(value="Dev", description="Dev", display="flex-shrink", layout=Layout(width="95%", align_self="center")).add_class("space"),
                 fourth,
                 widgets.Label(value="Decay", description="Decay", display="flex-shrink", layout=Layout(width="95%", align_self="center")).add_class("space"),
                 fifth,
                 filter_btn]).add_class('vertical-line')

left_box.layout = left_layout

right_box = VBox(children=[toggle_filter, emExibicao, toggle_cell]).add_class('horizontal-line')
right_box.layout=right_layout

mainBox = VBox(children=[])
mainBox.layout=main_layout

vBox = VBox(children=[right_box, mainBox])
vBox.layout = vBox_layout

# print left_box.layout.keys

box = Box(children=[left_box, vBox])
box.layout = box_layout

display(HTML(css))
box

ERROR:root:File `u'modulo_busca_arquivos.py'` not found.


NameError: name 't' is not defined

In [18]:
for i in dev:
if i.value:
    print i.description

IndentationError: expected an indented block (<ipython-input-18-49d6939f8852>, line 2)